In [5]:
import numpy as np
import functools as ft

# Basis states

In [6]:
pauli_Z = np.array([1, 0, 0, -1]).reshape([2, 2])
pauli_X = np.array([0, 1, 1, 0]).reshape([2, 2])
pauli_Y = np.array([0, -1j, 1j, 0]).reshape([2, 2])

_eigenvalues, _eigenvectors = np.linalg.eig(pauli_Z)
H = _eigenvectors[0]
V = _eigenvectors[1]

_eigenvalues, _eigenvectors = np.linalg.eig(pauli_X)
D = _eigenvectors[0]
A = _eigenvectors[1]

_eigenvalues, _eigenvectors = np.linalg.eig(pauli_Y)
R = _eigenvectors[0]
L = _eigenvectors[1]

In [53]:
def densitymatrix(state):
    return np.outer(state, np.conjugate(state))

In [54]:
densitymatrix(Bell["Psi Minus"])

array([[ 0. ,  0. , -0. ,  0. ],
       [ 0. ,  0.5, -0.5,  0. ],
       [-0. , -0.5,  0.5, -0. ],
       [ 0. ,  0. , -0. ,  0. ]])

In [38]:
Bell["Psi Minus"]

array([ 0.        ,  0.70710678, -0.70710678,  0.        ])

Below is an example of how to call a function that acts iteratively on the arguments, similar to how mapping works in mathematica.

In [7]:
args = [np.arange(2), np.arange(2,4)]
np.kron(*args)

array([0, 0, 2, 3])

Could write something clever for calling the Kron of basis vectors. Need to make the basis vectors a dictionary, in order to call something from a string. Think thats the cleanest way to go, rather than bulk editting HH, HV, VH, VV, DD, DA etc...

In [8]:
Bell = {
    "Psi Minus": (1/np.sqrt(2))*(np.kron(H,V) - np.kron(V,H)),
    "Psi Plus": (1/np.sqrt(2))*(np.kron(H,V) + np.kron(V,H)),
    "Phi Plus": (1/np.sqrt(2))*(np.kron(H,H) + np.kron(V,V)),
    "Phi Minus": (1/np.sqrt(2))*(np.kron(H,H) - np.kron(V,V))
}

In [9]:
Bell["Psi Minus"]

array([ 0.        ,  0.70710678, -0.70710678,  0.        ])

In [10]:
np.kron(H,H)

array([1., 0., 0., 0.])

In [11]:
_args = [H, H]
np.kron(*_args)

array([1., 0., 0., 0.])

Writing a function which can perform the Kronecker delta on more than two arguments, run through a quick example first.

In [12]:
_args = [H, H, H, H]

ft.reduce(lambda x, y: np.kron(x, y), _args)

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

The ft.reduce function is roughly equivalent to the following lines of code. Additionally, I define kronk, which will take the Kronecker product of a list of inputs

In [13]:
def reduce(function, iterable, initializer = None):
    it = iter(iterable)
    if initializer is None:
        value = next(it)
    else:
        value = initializer
    for element in it:
        value = function(value, element)
    return value

def kronk(iterable):
    return reduce(np.kron, iterable)

In [14]:
reduce(np.kron, _args)

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
kronk([H,V,H])

array([0., 0., 1., 0., 0., 0., 0., 0.])

The below generates a GHZ state of arbitrary dimensions by calling GHZ(n) where n is the dimension of the state

In [16]:
def GHZ(dimension):

    _basis_vector_H = []
    _basis_vector_V = []
    _basis_vectors = [H, V]
    
    for i in range(dimension):
        _basis_vector_H.append(_basis_vectors[0])
        _basis_vector_V.append(_basis_vectors[1])

    
    return 1/np.sqrt(2)*(reduce(np.kron, _basis_vector_H) + reduce(np.kron, _basis_vector_V))


In [17]:
GHZ(4)

array([0.70710678, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.70710678])

# Noise

In [18]:
def depol_noise(_rho, p):
    # only works for single qubit sub-space 
    # to scale, there needs to be an option to which qubit the noise is being applied to
    # once this option has been declared, there needs to be appropriate kronecker products made between identity and pauli operator 
    _rho_prime = (1 - p)*_rho + (p/3)*(pauli_X*_rho*pauli_X + pauli_Y*_rho*pauli_Y + pauli_Z*_rho*pauli_Z)
    return _rho_prime

In [59]:
rho = densitymatrix(H)

In [60]:
depol_noise(rho, 0.5)

array([[0.66666667+0.j, 0.        +0.j],
       [0.        +0.j, 0.        +0.j]])